# operations

In [19]:
class Operation():
    
    def __init__(self,input_nodes=[]):
        self.input_nodes = input_nodes
        self.output_nodes = []
        
        for node in input_nodes:
            node.output_nodes.append(self)
            
            
        _default_graph.operations.append(self)
    
    def compute(self):
        pass

In [20]:
class add(Operation):
    
    def __init__(self,x,y):
        super().__init__([x,y])
        
        
    def compute(self,x_var,y_var):
        self.inputs = [x_var,y_var]
        return x_var + y_var

In [21]:
class multiply(Operation):
    
    def __init__(self,x,y):
        super().__init__([x,y])
        
        
    def compute(self,x_var,y_var):
        self.inputs = [x_var,y_var]
        return x_var * y_var

In [28]:
class matmul(Operation):
    
    def __init__(self,x,y):
        super().__init__([x,y])
        
        
    def compute(self,x_var,y_var):
        
        self.inputs = [x_var,y_var]
        return x_var.dot(y_var)

In [23]:
class Placeholder():
    
    def __init__(self):
        
        self.output_nodes = []
        
        _default_graph.placeholders.append(self)

In [29]:
class Variable():
    
    def __init__(self,initial_value = None):
        
        self.value = initial_value
        self.output_nodes = []
        
        _default_graph.variables.append(self)
    

In [30]:
class Graph():
    
    def __init__(self):
        
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_default(self):
        
        global _default_graph
        _default_graph = self

In [31]:
g = Graph()

In [32]:
g.set_as_default()

In [33]:
A = Variable(10)

In [34]:
b = Variable(1)

In [35]:
x = Placeholder()

In [36]:
y = multiply(A,x)

In [37]:
z = add(y,b)

In [38]:
import numpy as np

In [39]:
def traverse_postorder(operation):

    
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)

    recurse(operation)
    return nodes_postorder

In [43]:
class Session:
    
    def run(self, operation, feed_dict = {}):
        """ 
          operation: The operation to compute
          feed_dict: Dictionary mapping placeholders to input values (the data)  
        """
        
        # Puts nodes in correct order
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:

            if type(node) == Placeholder:
                
                node.output = feed_dict[node]
                
            elif type(node) == Variable:
                
                node.output = node.value
                
            else: # Operation
                
                node.inputs = [input_node.output for input_node in node.input_nodes]

                 
                node.output = node.compute(*node.inputs)
                
            # Convert lists to numpy arrays
            if type(node.output) == list:
                node.output = np.array(node.output)
        
        # Return the requested node value
        return operation.output
